# OIDC Device Flow Authentication for JupyterLite

This notebook demonstrates how to authenticate with the Mat3ra API using OIDC Device Code Flow.

## 📖 First Time Using This Notebook?

**Quick visual guide**: [VISUAL_GUIDE.md](./VISUAL_GUIDE.md) - See exactly what each screen looks like!

**Detailed walkthrough**: [AUTHENTICATION_FLOW.md](./AUTHENTICATION_FLOW.md)

These guides explain:
- ✅ What happens when you click the authorization button
- ✅ What the login page looks like (with ASCII art examples!)
- ✅ Step-by-step expectations for each screen
- ✅ Common issues and solutions
- ✅ Timeline: How long each step takes

**TL;DR**: Click the green button → Login → Enter code → Approve → Done!

## Prerequisites

Before running this notebook, ensure:

1. **✅ OIDC Server is Running**: 
   ```bash
   ./run-meteor.sh -p=3000 -d=local
   ```
   Wait for: `App running at: http://localhost:3000/`

2. **✅ You have a user account**: 
   - Either already created at http://localhost:3000
   - Or you'll create one during the authentication flow

3. **✅ Default credentials are configured** (already set up in your app):
   - Client ID: `default-client`
   - Client Secret: `default-secret`
   - These are pre-configured in `src/application/settings.json`

### What Will Happen

When you run authentication:
1. **A green button appears** → Click it to open the login page
2. **Login page opens** → Login with your credentials (or create account)
3. **Enter device code** → Copy/paste the code shown in the notebook
4. **Approve authorization** → Grant access to your account
5. **Token received** → Notebook automatically gets your access token

## Device Code Flow Overview

```
┌─────────┐                              ┌──────────┐
│ Jupyter │                              │   OIDC   │
│ Notebook│                              │  Server  │
└────┬────┘                              └─────┬────┘
     │                                         │
     │ 1. Request device code                 │
     │────────────────────────────────────────>│
     │                                         │
     │ 2. Return device_code & user_code      │
     │<────────────────────────────────────────│
     │                                         │
     │ 3. Display URL & code to user          │
     │                                         │
┌────┴────┐                              ┌─────┴────┐
│  User   │                              │   OIDC   │
│ Browser │                              │  Server  │
└────┬────┘                              └─────┬────┘
     │                                         │
     │ 4. Navigate to URL, enter code         │
     │────────────────────────────────────────>│
     │                                         │
     │ 5. Login & authorize                   │
     │<───────────────────────────────────────>│
     │                                         │
┌────┴────┐                              ┌─────┴────┐
│ Jupyter │                              │   OIDC   │
│ Notebook│                              │  Server  │
└────┬────┘                              └─────┬────┘
     │                                         │
     │ 6. Poll for token                      │
     │────────────────────────────────────────>│
     │                                         │
     │ 7. Return access_token                 │
     │<────────────────────────────────────────│
     │                                         │
```

## Configuration

Update these settings to match your environment:

In [ ]:
# OIDC Configuration
OIDC_BASE_URL = "http://localhost:3000/oidc"  # Your OIDC server URL
CLIENT_ID = "default-client"                  # Your OAuth client ID
CLIENT_SECRET = "default-secret"              # Your OAuth client secret
SCOPE = "openid profile email"                # Requested scopes

print("✓ Configuration loaded")
print(f"  OIDC URL: {OIDC_BASE_URL}")
print(f"  Client ID: {CLIENT_ID}")

## Install Dependencies

Install required packages for HTTP requests:

## ⚠️ Common Issue: Client Authentication Failed

If you see `"error":"invalid_client","error_description":"client authentication failed"`, it means the OIDC clients haven't been loaded into the database yet.

**Quick Fix:**
```bash
cd src/application
meteor mongo < ../documentation/oidc/fix-oidc-clients.js
# Then restart the app
```

**Detailed instructions**: See [FIX_CLIENT_AUTH.md](./FIX_CLIENT_AUTH.md)

## Pre-Flight Check

Let's verify the OIDC server is running before we attempt authentication:

In [ ]:
import requests

def check_server_status():
    """Check if the OIDC server is running and accessible."""
    base_url = OIDC_BASE_URL.replace('/oidc', '')
    
    print(f"🔍 Checking server status...")
    print(f"   Server URL: {base_url}")
    
    try:
        # Check if server is responding
        response = requests.get(f"{base_url}/healthcheck", timeout=5)
        
        if response.status_code == 200:
            print(f"✅ Server is running and responding!")
            print(f"   Status: OK")
            
            # Check OIDC discovery endpoint
            try:
                oidc_response = requests.get(
                    f"{OIDC_BASE_URL}/.well-known/openid-configuration",
                    timeout=5
                )
                if oidc_response.status_code == 200:
                    config = oidc_response.json()
                    print(f"✅ OIDC server is configured!")
                    print(f"   Issuer: {config.get('issuer', 'N/A')}")
                    print(f"   Device endpoint: {config.get('device_authorization_endpoint', 'N/A')}")
                    return True
                else:
                    print(f"⚠️  OIDC endpoints not accessible")
                    return False
            except Exception as e:
                print(f"⚠️  OIDC discovery failed: {e}")
                return False
        else:
            print(f"⚠️  Server responded with status: {response.status_code}")
            return False
            
    except requests.exceptions.ConnectionError:
        print(f"❌ Cannot connect to server at {base_url}")
        print(f"\n💡 To start the server, run:")
        print(f"   cd /Users/mat3ra/code/GREEN/stack/web-app")
        print(f"   ./run-meteor.sh -p=3000 -d=local")
        print(f"\n   Wait for: 'App running at: http://localhost:3000/'")
        return False
    except requests.exceptions.Timeout:
        print(f"❌ Server connection timed out")
        print(f"   The server might be starting up. Wait a moment and try again.")
        return False
    except Exception as e:
        print(f"❌ Error checking server: {e}")
        return False

# Run the check
server_ok = check_server_status()

if server_ok:
    print(f"\n✨ Ready to authenticate!")
else:
    print(f"\n⏸️  Please start the server first, then run this cell again.")

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install("requests")
    print("✓ Dependencies installed")

## Authentication Function

This function implements the complete device code flow:

In [ ]:
import time
import os
import requests
from IPython.display import HTML, display, clear_output


async def authenticate_device_flow(
    oidc_base_url=OIDC_BASE_URL,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope=SCOPE,
):
    """
    Authenticate using OIDC Device Code Flow.
    
    Returns:
        dict: Token response containing access_token, token_type, expires_in, etc.
    
    Raises:
        Exception: If authentication fails
    """
    
    # Step 1: Request device code
    print("📡 Requesting device code from server...")
    print(f"   Connecting to: {oidc_base_url}/device/auth")
    
    try:
        device_response = requests.post(
            f"{oidc_base_url}/device/auth",
            data={
                "client_id": client_id,
                "client_secret": client_secret,
                "scope": scope,
            },
            headers={"Content-Type": "application/x-www-form-urlencoded"},
            timeout=10,
        )
    except requests.exceptions.RequestException as e:
        error_msg = str(e)
        if "Failed to execute 'send' on 'XMLHttpRequest'" in error_msg or "TimeoutError" in error_msg:
            raise Exception(
                f"⚠️ Cannot connect to OIDC server at {oidc_base_url}\n\n"
                f"Please make sure:\n"
                f"  1. The Mat3ra application is running: ./run-meteor.sh -p=3000 -d=local\n"
                f"  2. The server has finished starting (wait for 'App running at' message)\n"
                f"  3. You can access http://localhost:3000 in your browser\n\n"
                f"Error: {error_msg}"
            )
        raise Exception(f"Network error connecting to OIDC server: {e}")
    
    if device_response.status_code != 200:
        error_data = device_response.json() if device_response.headers.get('content-type', '').startswith('application/json') else {}
        error_msg = error_data.get("error_description") or error_data.get("error") or device_response.text
        
        if "invalid_client" in error_msg.lower():
            raise Exception(
                f"Client authentication failed. Please check:\n"
                f"  1. CLIENT_ID and CLIENT_SECRET are correct\n"
                f"  2. The OIDC client is configured in your application\n"
                f"  3. The client has device_code grant type enabled\n"
                f"\nError: {error_msg}"
            )
        else:
            raise Exception(f"Failed to get device code: {error_msg}")
    
    device_data = device_response.json()
    device_code = device_data["device_code"]
    user_code = device_data["user_code"]
    verification_uri = device_data["verification_uri"]
    verification_uri_complete = device_data.get("verification_uri_complete", verification_uri)
    interval = device_data.get("interval", 5)
    expires_in = device_data.get("expires_in", 600)
    
    print(f"✓ Device code received!")
    print(f"✓ User code: {user_code}")
    
    # Step 2: Display authorization instructions
    clear_output(wait=True)
    display(HTML(f'''
    <div style="padding: 20px; border: 3px solid #2196F3; border-radius: 10px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; font-family: Arial, sans-serif; box-shadow: 0 8px 16px rgba(0,0,0,0.3);">
        <h2 style="margin: 0 0 20px 0;">🔐 Device Flow Authentication</h2>
        
        <div style="background-color: rgba(255, 255, 255, 0.95); padding: 20px; border-radius: 8px; margin-bottom: 15px; color: #333;">
            <h3 style="margin: 0 0 15px 0; color: #2196F3;">📋 What Will Happen:</h3>
            <ol style="margin: 0; padding-left: 20px; line-height: 2;">
                <li><strong>Click the button below</strong> → Opens login/authorization page</li>
                <li><strong>Login to your account</strong> → If you're not already logged in</li>
                <li><strong>Enter the code shown below</strong> → Verifies this is your device</li>
                <li><strong>Approve the authorization</strong> → Grants access to your account</li>
                <li><strong>Return here</strong> → Notebook will automatically receive your access token</li>
            </ol>
        </div>
        
        <div style="background-color: rgba(255, 255, 255, 0.1); padding: 20px; border-radius: 8px; margin-bottom: 15px; text-align: center;">
            <h3 style="margin: 0 0 15px 0;">👉 Step 1: Open Login & Authorization Page</h3>
            <a href="{verification_uri_complete}" 
               target="_blank" 
               style="display: inline-block; font-size: 18px; padding: 15px 40px; background-color: #4CAF50; color: white; text-decoration: none; border-radius: 8px; font-weight: bold; box-shadow: 0 4px 6px rgba(0,0,0,0.3); transition: all 0.3s; text-transform: uppercase;">
                🚀 Click Here to Login & Authorize
            </a>
            <p style="margin: 10px 0 0 0; font-size: 13px; opacity: 0.9;">This opens a new window/tab - You may need to allow popups</p>
        </div>
        
        <div style="background-color: rgba(255, 255, 255, 0.1); padding: 20px; border-radius: 8px; margin-bottom: 15px; text-align: center;">
            <h3 style="margin: 0 0 10px 0;">🔑 Step 2: Enter This Code on the Page:</h3>
            <div style="font-size: 40px; font-weight: bold; letter-spacing: 8px; background-color: white; color: #2196F3; padding: 25px; border-radius: 8px; text-align: center; font-family: 'Courier New', monospace; box-shadow: 0 4px 6px rgba(0,0,0,0.2);">
                {user_code}
            </div>
            <p style="margin: 10px 0 0 0; font-size: 14px; opacity: 0.9;">Copy this code and paste it when prompted</p>
        </div>
        
        <div style="background-color: rgba(255, 200, 87, 0.25); padding: 15px; border-radius: 8px; border-left: 4px solid #FFC107;">
            <p style="margin: 0; font-size: 14px;">⏱️ <strong>Time remaining:</strong> {expires_in} seconds ({expires_in // 60} minutes)</p>
            <p style="margin: 8px 0 0 0; font-size: 14px;">⏳ <strong>Status:</strong> Waiting for you to login and authorize...</p>
        </div>
    </div>
    '''))
    
    # Step 3: Poll for token
    start_time = time.time()
    poll_count = 0
    
    while time.time() - start_time < expires_in:
        poll_count += 1
        
        try:
            token_response = requests.post(
                f"{oidc_base_url}/token",
                data={
                    "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
                    "device_code": device_code,
                    "client_id": client_id,
                    "client_secret": client_secret,
                },
                headers={"Content-Type": "application/x-www-form-urlencoded"},
                timeout=10,
            )
            
            # Success!
            if token_response.status_code == 200:
                token_data = token_response.json()
                
                # Store tokens in environment variables
                os.environ['OIDC_ACCESS_TOKEN'] = token_data["access_token"]
                if "refresh_token" in token_data:
                    os.environ['OIDC_REFRESH_TOKEN'] = token_data["refresh_token"]
                
                # Display success message
                clear_output(wait=True)
                display(HTML('''
                <div style="padding: 20px; border: 3px solid #4CAF50; border-radius: 10px; background-color: #d4edda; font-family: Arial, sans-serif;">
                    <h2 style="color: #155724; margin: 0 0 10px 0;">✅ Authentication Successful!</h2>
                    <p style="color: #155724; margin: 0; font-size: 16px;">You can now use the API with your access token.</p>
                    <p style="color: #155724; margin: 10px 0 0 0; font-size: 14px;">Access token is stored in: <code>os.environ['OIDC_ACCESS_TOKEN']</code></p>
                </div>
                '''))
                
                return token_data
            
            # Handle error responses
            error_data = token_response.json() if token_response.headers.get('content-type', '').startswith('application/json') else {}
            error = error_data.get("error", "")
            error_description = error_data.get("error_description", "")
            error_msg = error_description or error or token_response.text
            
            # Check if authorization is still pending
            if (
                error == "authorization_pending" or
                error == "slow_down" or
                "authorization_pending" in error_msg.lower() or
                "slow_down" in error_msg.lower() or
                "authorization request is still pending" in error_msg.lower()
            ):
                # Still waiting for user authorization
                if error == "slow_down":
                    interval += 5  # Increase polling interval
                
                # Update status
                if poll_count % 3 == 0:  # Update display every 3 polls
                    elapsed = int(time.time() - start_time)
                    remaining = expires_in - elapsed
                    print(f"⏳ Still waiting... ({elapsed}s elapsed, {remaining}s remaining)", end='\r')
                
                time.sleep(interval)
                continue
            
            # Handle other errors
            elif error == "expired_token":
                raise Exception("Device code expired. Please run the authentication again.")
            elif error == "access_denied":
                raise Exception("Authorization denied by user.")
            else:
                raise Exception(f"Unexpected error: {error_msg}")
                
        except requests.exceptions.RequestException as e:
            print(f"\n⚠️ Network error during polling: {e}")
            time.sleep(interval)
            continue
    
    # Timeout
    raise Exception(f"Timeout waiting for authorization ({expires_in}s). Please try again.")


print("✓ Authentication function defined")

## Run Authentication

**Important**: Before running this cell:
1. Make sure the server status check above passed ✅
2. Keep your browser ready - a login page will open automatically
3. Have your account credentials ready

When you run this cell:
- A colorful authorization box will appear with a **green button**
- **Click the green button** to open the login/authorization page
- **Login** if you're not already logged in
- **Enter the code** shown in the box
- **Approve** the authorization
- Come back here and wait - the token will appear automatically!

Execute the device flow authentication:

In [ ]:
try:
    tokens = await authenticate_device_flow()
    
    print("\n📋 Token Information:")
    print(f"  Token Type: {tokens.get('token_type', 'N/A')}")
    print(f"  Expires In: {tokens.get('expires_in', 'N/A')} seconds")
    print(f"  Access Token (first 50 chars): {tokens['access_token'][:50]}...")
    if 'refresh_token' in tokens:
        print(f"  Refresh Token: Available")
    
except Exception as e:
    print(f"\n❌ Authentication failed: {e}")

## Using the Access Token

Now you can use the access token to make authenticated API requests:

In [ ]:
def call_api(endpoint, method="GET", data=None):
    """
    Make an authenticated API request.
    
    Args:
        endpoint: API endpoint (e.g., '/api/v1/users/me')
        method: HTTP method (GET, POST, PUT, DELETE)
        data: Request body for POST/PUT requests
    
    Returns:
        Response data
    """
    access_token = os.environ.get('OIDC_ACCESS_TOKEN')
    
    if not access_token:
        raise Exception("No access token found. Please authenticate first.")
    
    # Construct full URL
    base_url = OIDC_BASE_URL.replace('/oidc', '')  # Remove /oidc suffix
    url = f"{base_url}{endpoint}"
    
    # Make request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }
    
    response = requests.request(
        method=method,
        url=url,
        headers=headers,
        json=data,
        timeout=30,
    )
    
    if response.status_code >= 400:
        raise Exception(f"API request failed ({response.status_code}): {response.text}")
    
    return response.json() if response.headers.get('content-type', '').startswith('application/json') else response.text


print("✓ API helper function defined")

### Example: Get Current User Info

In [ ]:
try:
    user_info = call_api('/api/v1/users/me')
    print("👤 User Information:")
    print(user_info)
except Exception as e:
    print(f"❌ Error: {e}")

### Example: List Projects

In [ ]:
try:
    projects = call_api('/api/v1/projects')
    print(f"📁 Projects ({len(projects)} total):")
    for project in projects[:5]:  # Show first 5
        print(f"  - {project.get('name', 'Unnamed')} (ID: {project.get('_id', 'N/A')})")
except Exception as e:
    print(f"❌ Error: {e}")

## Token Management

Check token status and refresh if needed:

In [ ]:
def check_token_status():
    """Check if we have a valid access token."""
    access_token = os.environ.get('OIDC_ACCESS_TOKEN')
    refresh_token = os.environ.get('OIDC_REFRESH_TOKEN')
    
    print("🔑 Token Status:")
    print(f"  Access Token: {'✓ Present' if access_token else '✗ Missing'}")
    print(f"  Refresh Token: {'✓ Present' if refresh_token else '✗ Missing'}")
    
    if access_token:
        print(f"  Access Token (first 30 chars): {access_token[:30]}...")
    
    return bool(access_token)


def clear_tokens():
    """Clear stored tokens."""
    os.environ.pop('OIDC_ACCESS_TOKEN', None)
    os.environ.pop('OIDC_REFRESH_TOKEN', None)
    print("✓ Tokens cleared")


check_token_status()